In [1]:
# import required libraries
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import pickle
import joblib

In [2]:
# Load the prediction transactions sheet in pandas dataframe
dfp = pd.read_excel(r'C:\Users\pravi\Documents\Study\HSBC RC Hackathon 2025\Hackathon-2025-dev\Vikash\predict_transactions.xlsx')

In [3]:
dfp.head()

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date
0,Jackson Group,Saint Helena,Hill-Kelly PLC,Guatemala,48455.62,USD,2025-08-25
1,Davis-Bennett PLC,Cape Verde,Wright Partners Ltd,Equatorial Guinea,27499.36,USD,2025-06-04
2,"Stewart, Brown and Jackson LLC",Iceland,Ruiz Holdings Company,Central African Republic,7072.33,USD,2025-04-13
3,Turner Group,Djibouti,Mendoza Partners Corporation,Cameroon,8039.73,USD,2025-08-04
4,"Stewart, Young and White Ltd",Zambia,"Mitchell, Garcia and Lewis LLC",Thailand,14394.15,USD,2025-02-06


In [4]:
dfp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   originator_company   402 non-null    object 
 1   originator_country   402 non-null    object 
 2   beneficiary_company  402 non-null    object 
 3   beneficiary_country  402 non-null    object 
 4   amount               402 non-null    float64
 5   currency             402 non-null    object 
 6   date                 402 non-null    object 
dtypes: float64(1), object(6)
memory usage: 22.1+ KB


In [5]:
# Load references sheet into dataframe
dfr = pd.read_csv(r'C:\Users\pravi\Documents\Study\HSBC RC Hackathon 2025\Hackathon-2025-dev\Vikash\company_mapping_per_gemini.csv')

In [6]:
dfr.head()

,company_name,country,sic_code,sic_description
0,Watson Industries and Sons,Guinea-Bissau,6719,"Offices of Holding Companies, Not Elsewhere Cl..."
1,"Foster, Mitchell and Stewart Company",Somalia,3714,Motor Vehicle Parts and Accessories
2,Garcia-Parker Group,Morocco,5162,Plastics Materials and Basic Forms and Shapes
3,"Taylor, Smith and Richardson Ltd",Tuvalu,3341,Secondary Smelting and Refining of Nonferrous ...
4,"Richardson, Cooper and Alvarez and Sons",Belize,7216,"Drycleaning Plants, Except Rug Cleaning"


In [7]:
dfr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company_name     800 non-null    object
 1   country          800 non-null    object
 2   sic_code         800 non-null    int64 
 3   sic_description  800 non-null    object
dtypes: int64(1), object(3)
memory usage: 25.1+ KB


In [9]:
# Extract SIC code for an originator and beneficiary based on fuzzy logic on name and exact logic on country
# https://medium.com/@kasperjuunge/rapidfuzz-explained-c26e93b6012d

def fuzzy_match_id(name, country, df_ref, scorer=fuzz.WRatio, score_cutoff=80):
    if pd.isna(name) or pd.isna(country):
        return pd.NA, pd.NA
    
    # Filter df_ref by exact country match
    candidates = dfr[dfr['country'] == country]
    if candidates.empty:
        return pd.NA, pd.NA
    
    # Extract list of candidate names
    candidate_names = candidates['company_name'].tolist()
    
    # Fuzzy match the name against candidate names
    match = process.extractOne(name, candidate_names, scorer=scorer, score_cutoff=score_cutoff)
    if match:
        best_name, score, _ = match
        matched_row = candidates[candidates['company_name'] == best_name].iloc[0]
        matched_id = matched_row['sic_code']
        description = matched_row.get('sic_description', pd.NA)  # Use pd.NA if Description is missing
        return matched_id, description
    else:
        return pd.NA, pd.NA

# Apply fuzzy match lookup for originator
dfp[['originator_id','originator_desc']] = dfp.apply(
    lambda row: pd.Series(fuzzy_match_id(row['originator_company'], row['originator_country'], dfr)), axis=1)

# Apply fuzzy match lookup for beneficiary
dfp[['beneficiary_id','beneficiary_desc']] = dfp.apply(
    lambda row: pd.Series(fuzzy_match_id(row['beneficiary_company'], row['beneficiary_country'], dfr)), axis=1)


In [10]:
dfp.head()

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date,originator_id,originator_desc,beneficiary_id,beneficiary_desc
0,Jackson Group,Saint Helena,Hill-Kelly PLC,Guatemala,48455.62,USD,2025-08-25,7322,Adjustment and Collection Services,131,Cotton
1,Davis-Bennett PLC,Cape Verde,Wright Partners Ltd,Equatorial Guinea,27499.36,USD,2025-06-04,2441,Nailed and Lock Corner Wood Boxes and Shook,5145,Confectionery
2,"Stewart, Brown and Jackson LLC",Iceland,Ruiz Holdings Company,Central African Republic,7072.33,USD,2025-04-13,7699,"Repair Shops and Related Services, Not Elsewhe...",3915,"Jewelers' Findings and Materials, and Lapidary..."
3,Turner Group,Djibouti,Mendoza Partners Corporation,Cameroon,8039.73,USD,2025-08-04,171,Berry Crops,2436,Softwood Veneer and Plywood
4,"Stewart, Young and White Ltd",Zambia,"Mitchell, Garcia and Lewis LLC",Thailand,14394.15,USD,2025-02-06,5113,Industrial and Personal Service Paper,2295,"Coated Fabrics, Not Rubberized"


In [11]:
dfp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   originator_company   402 non-null    object 
 1   originator_country   402 non-null    object 
 2   beneficiary_company  402 non-null    object 
 3   beneficiary_country  402 non-null    object 
 4   amount               402 non-null    float64
 5   currency             402 non-null    object 
 6   date                 402 non-null    object 
 7   originator_id        402 non-null    int64  
 8   originator_desc      402 non-null    object 
 9   beneficiary_id       402 non-null    int64  
 10  beneficiary_desc     402 non-null    object 
dtypes: float64(1), int64(2), object(8)
memory usage: 34.7+ KB


In [12]:
# Defining features and target
features = ["originator_id", "beneficiary_id", "amount_log", "same_industry"]

In [13]:
# Create required features
dfp["same_industry"] = (dfp["originator_id"] == dfp["beneficiary_id"]).astype(int)
dfp["amount_log"] = np.log1p(dfp["amount"])

In [14]:
dfp.head()

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date,originator_id,originator_desc,beneficiary_id,beneficiary_desc,same_industry,amount_log
0,Jackson Group,Saint Helena,Hill-Kelly PLC,Guatemala,48455.62,USD,2025-08-25,7322,Adjustment and Collection Services,131,Cotton,0,10.788424
1,Davis-Bennett PLC,Cape Verde,Wright Partners Ltd,Equatorial Guinea,27499.36,USD,2025-06-04,2441,Nailed and Lock Corner Wood Boxes and Shook,5145,Confectionery,0,10.221954
2,"Stewart, Brown and Jackson LLC",Iceland,Ruiz Holdings Company,Central African Republic,7072.33,USD,2025-04-13,7699,"Repair Shops and Related Services, Not Elsewhe...",3915,"Jewelers' Findings and Materials, and Lapidary...",0,8.864087
3,Turner Group,Djibouti,Mendoza Partners Corporation,Cameroon,8039.73,USD,2025-08-04,171,Berry Crops,2436,Softwood Veneer and Plywood,0,8.992275
4,"Stewart, Young and White Ltd",Zambia,"Mitchell, Garcia and Lewis LLC",Thailand,14394.15,USD,2025-02-06,5113,Industrial and Personal Service Paper,2295,"Coated Fabrics, Not Rubberized",0,9.574647


In [15]:
dfp_f = dfp[features]

In [16]:
dfp_f.head()

,originator_id,beneficiary_id,amount_log,same_industry
0,7322,131,10.788424,0
1,2441,5145,10.221954,0
2,7699,3915,8.864087,0
3,171,2436,8.992275,0
4,5113,2295,9.574647,0


In [17]:
dfp_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   originator_id   402 non-null    int64  
 1   beneficiary_id  402 non-null    int64  
 2   amount_log      402 non-null    float64
 3   same_industry   402 non-null    int32  
dtypes: float64(1), int32(1), int64(2)
memory usage: 11.1 KB


In [19]:
# Load the model from the .pkl file
loaded_model = joblib.load('C:\\Users\\pravi\\Documents\\Study\\HSBC RC Hackathon 2025\\TransactionAnalysis_mlp.pkl')

In [20]:
# Use the loaded model to make predictions
predictions = loaded_model.predict(dfp_f)

In [21]:
dfp['validity_pred'] = predictions

In [22]:
dfp.head()

,originator_company,originator_country,beneficiary_company,beneficiary_country,amount,currency,date,originator_id,originator_desc,beneficiary_id,beneficiary_desc,same_industry,amount_log,validity_pred
0,Jackson Group,Saint Helena,Hill-Kelly PLC,Guatemala,48455.62,USD,2025-08-25,7322,Adjustment and Collection Services,131,Cotton,0,10.788424,0
1,Davis-Bennett PLC,Cape Verde,Wright Partners Ltd,Equatorial Guinea,27499.36,USD,2025-06-04,2441,Nailed and Lock Corner Wood Boxes and Shook,5145,Confectionery,0,10.221954,0
2,"Stewart, Brown and Jackson LLC",Iceland,Ruiz Holdings Company,Central African Republic,7072.33,USD,2025-04-13,7699,"Repair Shops and Related Services, Not Elsewhe...",3915,"Jewelers' Findings and Materials, and Lapidary...",0,8.864087,0
3,Turner Group,Djibouti,Mendoza Partners Corporation,Cameroon,8039.73,USD,2025-08-04,171,Berry Crops,2436,Softwood Veneer and Plywood,0,8.992275,1
4,"Stewart, Young and White Ltd",Zambia,"Mitchell, Garcia and Lewis LLC",Thailand,14394.15,USD,2025-02-06,5113,Industrial and Personal Service Paper,2295,"Coated Fabrics, Not Rubberized",0,9.574647,0


In [23]:
dfp["validity_pred"].value_counts()

validity_pred
0    214
1    188
Name: count, dtype: int64

In [25]:
# Save the DataFrame with the new 'prediction' column to a new Excel file

dfp.to_excel('C:\\Users\\pravi\\Documents\\Study\\HSBC RC Hackathon 2025\\Hackathon-2025-dev\\Vikash\\mlp_pred_output.xlsx', index=False)